## Game and Vectors data Study

In [2]:
import empowermentexploration.utils.data_handle as data_handle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [5]:
game_version = 'tinyalchemy'
vector_version = 'crawl300'

element_vectors = data_handle.get_wordvectors(game_version, vector_version)


In [11]:
print(element_vectors[0])

[-1.28955141e-01 -6.85449541e-02 -7.64038786e-02  2.22069249e-02
  4.75167744e-02 -1.46887498e-02 -1.15821317e-01  1.67637598e-02
  1.22868352e-01  5.35856560e-02 -5.59497476e-02  3.25776786e-02
  1.19875139e-02  6.07428662e-02  2.66896538e-03  6.98655322e-02
  2.05734260e-02 -6.64751455e-02  2.55052689e-02  1.65347643e-02
 -4.65277582e-03 -7.66445650e-03 -4.68234357e-04 -4.01621833e-02
 -2.65102144e-02  9.61902663e-02 -2.75670141e-02  2.99962349e-02
  2.50794198e-02 -1.02899551e-01 -3.68212648e-02  7.89834280e-03
 -1.01460457e-01  9.70172137e-02  4.05623391e-03 -5.80472220e-03
  6.45781979e-02 -3.83974127e-02 -1.44214138e-01  8.08790699e-02
 -2.41390150e-02 -1.58793479e-01  3.21111269e-02 -5.50986007e-02
  8.75098631e-04  6.93960814e-03  4.04989682e-02 -1.41772337e-03
 -7.43098045e-03 -4.14701663e-02 -1.07810218e-02 -2.35747397e-02
  6.14165142e-02 -1.10595478e-02 -2.52572894e-02  4.00397256e-02
  4.11121687e-03 -2.15586424e-02 -5.90586551e-02  4.12000827e-02
  2.69113537e-02 -3.78472

In [4]:
game_version = 'tinyalchemy'
vector_version = 'crawl300'
prediction_model = 0

data_table = data_handle.get_combination_table(game_version)
data_table = data_table.sample(frac=1)      # Returns a random sample from the whole dataframe. The whole thing here. Shuffling. 

#if self.prediction_model == 0:
if prediction_model != 1:
    data_table = data_table.drop_duplicates(subset=['first', 'second'])

data = data_table

# print info for user
print('\nGet feature and label vectors.')

parent_table = data_handle.get_parent_table(game_version)
combination_table = data_handle.get_combination_table(game_version, csv=False)

# get list of element vectors
element_vectors = data_handle.get_wordvectors(game_version, vector_version)

# for element prediction model, empowerment prediction model and train and val data: reduce data to only successful data
if prediction_model != 0 and (type == 0 or type == 2):
    data.query('success == 1', inplace=True)
data = data.values      # Reduces the data to only the combinations that are succesful (i.e = 1)

# initialize feature vector storage
combination_count = data.shape[0]       # All succesful Combinations count.  
vector_dim = element_vectors.shape[1]       # Length of Word Vector. Size of crawl300 in this case. 
feature_vectors = np.empty((2*combination_count,2*vector_dim))      # Create empty matrix with the 2*dimensions of above measures.

# initialize label vector storage
if prediction_model != 1:
    label_vector = np.empty(2*combination_count)
else:
    label_vector = np.empty((2*combination_count, vector_dim))

# initialize element index storage
indices = np.empty((2*combination_count,4))

# create combination features and get corresponding result vectors
for idx, combination in enumerate(data):
    combination_success = [int(x) for x in combination[:3]]  # I don't understand this!!
    result = combination[3]
    # print('For Idx:{}\nCombination_success: {}\nCombination[3]/result: {}'.format(idx, combination_success, result))  
    feature_vectors[idx,:] = np.concatenate([element_vectors[combination_success[0]], element_vectors[combination_success[1]]], axis = 0)
    feature_vectors[idx+combination_count,:] = np.concatenate([element_vectors[combination_success[1]], element_vectors[combination_success[0]]], axis = 0)
    if idx < 1:
        print('element_vectors[combination_success[0]: {}\n Shape: {}'.format(element_vectors[combination_success[0]], element_vectors[combination_success[0]].shape))
    # print('element_vectors[combination_success[0]: {}'.format(element_vectors[combination_success[0]]))
    
    """
    if idx < 10:
        print('For Idx {}\nfeature_vectors[idx,:]: \n{}'.format(idx, feature_vectors[idx,:]))
        print('feature_vectors[idx+combination_count,:]: \n{}'.format(feature_vectors[idx+combination_count,:]))
    """

    if prediction_model == 0:  # Means Link prediction. 
        label_vector[idx] = combination_success[2]
        label_vector[idx+combination_count] = combination_success[2]
    elif prediction_model ==2:     # Empowerment prediction model. 
        #empvalue of result

        if combination[0] in combination_table and combination[1] in combination_table[combination[0]]:
            result_elements = combination_table[combination[0]][combination[1]]
        else:
            result_elements = list()

        # initialize empowerment depending on how it is calculated
        empowerment = set()

        # calculate empowerment value iteratively for each result
        for r in result_elements:
            if r in parent_table:
                empowerment.update(parent_table[r])

        label_vector[idx] = len(empowerment)
        label_vector[idx+combination_count] = len(empowerment)
    else:
        if not np.isnan(result):
            label_vector[idx,:] = element_vectors[int(result)]
            label_vector[idx+combination_count,:] = element_vectors[int(result)]

    indices[idx,:] = combination
    indices[idx+combination_count,:] = combination

print('Final Idx value: {}'.format(idx))

print('\t\t... shape input vectors:', feature_vectors.shape)
print('\t\t... shape label vector:', label_vector.shape)

# return feature vectors and corresponding label and index vectors
print('SHAPES:  \nFeature Vector: {}\nLabel Vector: {}\nIndices: {}'.format(feature_vectors.shape, label_vector.shape, indices.shape))


Get feature and label vectors.
element_vectors[combination_success[0]: [-1.80743307e-01 -8.78148451e-02 -1.24113232e-01  1.44273434e-02
  9.33597907e-02 -6.15679808e-02 -1.28582194e-02  6.19259179e-02
  1.82161182e-01  1.29158348e-02 -1.07790157e-01 -3.16625386e-02
  1.12745762e-02  1.46464687e-02 -6.18088767e-02  5.71295097e-02
  7.41609707e-02 -3.30966003e-02  1.58038028e-02 -4.13473025e-02
  6.38863146e-02 -6.02229349e-02 -1.41759133e-02  5.44504151e-02
  1.39582381e-02  7.31943399e-02  3.76267582e-02 -5.34070097e-02
  1.42679755e-02  4.86143753e-02 -2.65943743e-02 -5.99331185e-02
  1.18107861e-02 -1.71525404e-03 -2.17112945e-03  7.36561641e-02
  6.42323941e-02 -4.35904823e-02 -1.94482118e-01  7.81182200e-02
 -3.41324694e-02 -2.01484680e-01  3.00973803e-02  4.29719836e-02
 -8.55029076e-02 -1.37073351e-02  3.21958140e-02  4.18761745e-02
 -1.07293241e-01 -1.79506559e-03 -6.39483631e-02 -3.46448645e-02
  7.74981230e-02  1.39244972e-03  1.65017098e-02  5.27731255e-02
  7.29077756e-02  

In [12]:
print(feature_vectors.shape)

(292140, 600)


In [15]:
combination_table = pd.read_csv('empowermentexploration/resources/littlealchemy/data/{}CombinationTable.csv'.format(game_version),
                                dtype={'first': int, 'second': int, 'success': int, 'result': int})

print(combination_table.shape)

(146070, 4)


## Studying Elements.json

In [15]:
import json

with open('empowermentexploration\\resources\\littlealchemy\\data\\raw\\elements.JSON', 'r') as file:
        elements_dict = json.load(file)

print('Length of dict: {}'.format(len(elements_dict)))
print('Elements datatype: {}'.format(type(elements_dict)))

Length of dict: 863
Elements datatype: <class 'dict'>


In [21]:
import json

with open('empowermentexploration/resources/littlealchemy/data/alchemy1Elements.json', 'r') as file:
        elements_dict = json.load(file)

print('Length of dict: {}'.format(len(elements_dict)))
print('Elements datatype: {}'.format(type(elements_dict)))

Length of dict: 540
Elements datatype: <class 'list'>


In [16]:
# Studying elements.json file.

import json
import os
import pprint

def get_json_info(data):
    if isinstance(data, dict):
        print(f"Top-level structure: Dictionary")
        print(f"Number of keys: {len(data)}")
        print(f"Keys: {', '.join(data.keys())}")
    elif isinstance(data, list):
        print(f"Top-level structure: List")
        print(f"Number of items: {len(data)}")
        if data:
            print(f"Type of first item: {type(data[0]).__name__}")
    else:
        print(f"Top-level structure: {type(data).__name__}")

def analyze_json(data, level=0):
    if isinstance(data, dict):
        print(f"{'  ' * level}Dictionary with {len(data)} key-value pairs")
        for key, value in data.items():
            print(f"{'  ' * (level+1)}Key: {key}")
            analyze_json(value, level+1)
    elif isinstance(data, list):
        print(f"{'  ' * level}List with {len(data)} items")
        if data:
            analyze_json(data[0], level+1)
    else:
        print(f"{'  ' * level}Value of type: {type(data).__name__}")

def json_file_info(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    print(f"File: {file_path}")
    print(f"File size: {os.path.getsize(file_path)} bytes")
    print("\nStructure:")
    analyze_json(data)
    print("\nOverview:")
    get_json_info(data)
    print("\nSample of the data:")
    pprint.pprint(data, depth=2, compact=True)

json_file_info('empowermentexploration\\resources\\littlealchemy\\data\\raw\\elements.JSON')

File: empowermentexploration\resources\littlealchemy\data\raw\elements.JSON
File size: 23240 bytes

Structure:
Dictionary with 863 key-value pairs
  Key: rain,smoke
  List with 1 items
    Value of type: str
  Key: rain,smog
  List with 1 items
    Value of type: str
  Key: bird,metal
  List with 1 items
    Value of type: str
  Key: bird,steel
  List with 1 items
    Value of type: str
  Key: clock,sound
  List with 1 items
    Value of type: str
  Key: fruit,time
  List with 1 items
    Value of type: str
  Key: juice,time
  List with 1 items
    Value of type: str
  Key: plant,water
  List with 1 items
    Value of type: str
  Key: ocean,plant
  List with 2 items
    Value of type: str
  Key: plant,sea
  List with 2 items
    Value of type: str
  Key: life,space
  List with 1 items
    Value of type: str
  Key: dust,human
  List with 1 items
    Value of type: str
  Key: lizard,swamp
  List with 1 items
    Value of type: str
  Key: lizard,river
  List with 1 items
    Value of type

# H5 file

In [5]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd

f = pd. read_hdf('empowermentexploration\\resources\\customgametree\\data\\alchemy1GametreeTable-data-crawl300.h5')

In [17]:
print("Groups:", list(f.keys()))

Groups: ['predSuccess', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '

In [51]:
f.shape

(146070, 542)

In [105]:
f.head(20)

predSuccess         0         1         2         3         4  \
first second                                                                  
0.0   0.0        0.999741  0.064150  0.002360  0.000871  0.004529  0.000474   
      1.0        0.999480  0.029920  0.002409  0.001856  0.002962  0.001089   
      2.0        0.999357  0.032367  0.004586  0.005961  0.005210  0.001398   
      3.0        0.999224  0.025951  0.003252  0.002757  0.012534  0.000836   
      4.0        0.984779  0.005491  0.000440  0.000565  0.000895  0.000957   
      5.0        0.002974  0.004982  0.001093  0.000519  0.001728  0.001051   
      6.0        0.067529  0.003563  0.001070  0.000286  0.000874  0.000651   
      7.0        0.977826  0.009945  0.003082  0.001500  0.001450  0.000646   
      8.0        0.922843  0.007365  0.000669  0.000362  0.000990  0.001373   
      9.0        0.213605  0.012764  0.000766  0.000335  0.001380  0.000369   
      10.0       0.493560  0.002581  0.000940  0.000540  0.000751  0.000812   
      11.0       0.166313  0.025312  0.003944  0.000808  0.002677  0.000486   
      12.0       0.385725  0.009962  0.001940  0.000437  0.003070  0.000385   
      13.0       0.977701  0.032574  0.001956  0.000903  0.002509  0.002814   
      14.0       0.234090  0.011333  0.001016  0.000331  0.001594  0.000304   
      15.0       0.956810  0.009473  0.006956  0.001745  0.002653  0.001206   
      16.0       0.694126  0.001885  0.001280  0.000250  0.000617  0.000277   
      17.0       0.960673  0.007179  0.000927  0.000484  0.001144  0.000389   
      18.0       0.979539  0.006169  0.000526  0.000854  0.001165  0.000414   
      19.0       0.084948  0.008939  0.001488  0.000292  0.001545  0.000429   

                     5         6         7         8  ...       531       532  \
first second                                          ...                       
0.0   0.0     0.000570  0.000410  0.003884  0.000947  ...  0.002314  0.000913   
      1.0     0.000253  0.000489  0.002964  0.002258  ...  0.001357  0.000225   
      2.0     0.000932  0.000258  0.004172  0.000983  ...  0.002636  0.001692   
      3.0     0.001123  0.000253  0.007078  0.001731  ...  0.004474  0.000938   
      4.0     0.000187  0.000131  0.000431  0.000190  ...  0.003364  0.000919   
      5.0     0.005158  0.000522  0.001365  0.001383  ...  0.000825  0.001368   
      6.0     0.000355  0.011543  0.001498  0.001077  ...  0.000752  0.000684   
      7.0     0.000428  0.000383  0.080870  0.001228  ...  0.000710  0.000441   
      8.0     0.000548  0.001016  0.002795  0.008815  ...  0.000744  0.000494   
      9.0     0.000231  0.000504  0.001042  0.000570  ...  0.000394  0.000252   
      10.0    0.002074  0.000876  0.001176  0.000817  ...  0.000651  0.000748   
      11.0    0.000482  0.000628  0.004340  0.000937  ...  0.001474  0.000557   
      12.0    0.001598  0.000751  0.001056  0.001107  ...  0.001552  0.000403   
      13.0    0.001037  0.000820  0.001314  0.000882  ...  0.001052  0.001970   
      14.0    0.000523  0.000590  0.000788  0.000502  ...  0.000461  0.000408   
      15.0    0.002722  0.000897  0.005484  0.000749  ...  0.000824  0.001680   
      16.0    0.000460  0.001713  0.000882  0.000456  ...  0.000543  0.000256   
      17.0    0.000523  0.000509  0.003825  0.000740  ...  0.000520  0.000399   
      18.0    0.000702  0.000383  0.001768  0.000937  ...  0.001120  0.000911   
      19.0    0.000696  0.000568  0.001181  0.000483  ...  0.000681  0.000565   

                   533       534       535       536       537       538  \
first second                                                               
0.0   0.0     0.000513  0.000679  0.000699  0.002766  0.000635  0.000554   
      1.0     0.000707  0.000421  0.000492  0.002376  0.001227  0.001030   
      2.0     0.000376  0.000981  0.003669  0.003666  0.001268  0.001431   
      3.0     0.000399  0.001052  0.001100  0.004242  0.000846  0.001079   
      4.0     0.000131  0.000865  0.

In [61]:
f.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 146070 entries, (0.0, 0.0) to (539.0, 539.0)
Columns: 542 entries, predSuccess to predResult
dtypes: float64(541), int32(1)
memory usage: 608.1 MB


In [62]:
f.describe()

,predSuccess,0,1,2,3,4,5,6,7,8,...,531,532,533,534,535,536,537,538,539,predResult
count,1.460700e+05,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,...,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000,146070.000000
mean,8.656499e-02,0.002508,0.001797,0.000723,0.001105,0.001187,0.000887,0.001701,0.001774,0.001277,...,0.001110,0.001143,0.000974,0.000847,0.000791,0.001068,0.002752,0.001557,0.002174,271.730136
std,2.011453e-01,0.003072,0.002118,0.000751,0.001139,0.012461,0.001885,0.019476,0.004024,0.001841,...,0.002232,0.003212,0.001996,0.003604,0.001432,0.003601,0.004024,0.006086,0.016715,152.867633
min,1.305636e-36,0.000013,0.000012,0.000012,0.000005,0.000007,0.000011,0.000003,0.000014,0.000013,...,0.000011,0.000008,0.000005,0.000003,0.000016,0.000005,0.000011,0.000012,0.000011,0.000000
25%,1.233983e-04,0.000920,0.000741,0.000351,0.000530,0.000406,0.000365,0.000385,0.000794,0.000596,...,0.000507,0.000452,0.000464,0.000391,0.000381,0.000484,0.000816,0.000565,0.000442,154.000000
50%,2.319150e-03,0.001495,0.001143,0.000495,0.000770,0.000567,0.000541,0.000552,0.001097,0.000874,...,0.000723,0.000668,0.000629,0.000549,0.000517,0.000687,0.001430,0.000876,0.000706,271.000000
75%,4.099688e-02,0.002832,0.002034,0.000812,0.001256,0.000878,0.000922,0.000915,0.001721,0.001453,...,0.001160,0.001143,0.000968,0.000838,0.000777,0.001081,0.002985,0.001578,0.001335,400.000000
max,9.999834e-01,0.070287,0.059873,0.029547,0.026602,0.810558,0.227260,0.736958,0.325978,0.380720,...,0.464397,0.529495,0.195548,0.420962,0.181758,0.612155,0.092145,0.715365,0.833470,539.000000


In [75]:
f.sample(n=5)       # A random sample of 5 rows from the dataframe. 

,,predSuccess,0,1,2,3,4,5,6,7,8,...,531,532,533,534,535,536,537,538,539,predResult
first,second,,,,,,,,,,,,,,,,,,,,,
431.0,470.0,0.000012,0.002261,0.000877,0.000255,0.002252,0.001331,0.000390,0.000479,0.001143,0.001670,...,0.000693,0.000521,0.000769,0.000685,0.000385,0.001094,0.000604,0.000941,0.000904,153
8.0,384.0,0.125635,0.000874,0.000718,0.000829,0.000605,0.001800,0.004156,0.003062,0.000838,0.014372,...,0.001906,0.000955,0.003319,0.000991,0.000490,0.000530,0.001777,0.002367,0.003763,179
27.0,470.0,0.000082,0.001071,0.000314,0.000263,0.001063,0.000297,0.001882,0.000836,0.001672,0.001574,...,0.000412,0.000944,0.000953,0.000372,0.000291,0.002577,0.003285,0.000582,0.001350,187
106.0,296.0,0.010342,0.000466,0.001443,0.000201,0.000225,0.000626,0.000391,0.000789,0.000204,0.000221,...,0.000831,0.000289,0.000350,0.000492,0.000241,0.000141,0.000217,0.000451,0.000207,434
86.0,474.0,0.143765,0.013945,0.000726,0.001626,0.004957,0.001332,0.000732,0.000274,0.004722,0.001375,...,0.000385,0.000683,0.001056,0.000837,0.000375,0.002896,0.001262,0.001265,0.000394,386


In [84]:
f.loc[:, ['predSuccess']]

predSuccess
first second              
0.0   0.0     9.997410e-01
      1.0     9.994800e-01
      2.0     9.993572e-01
      3.0     9.992245e-01
      4.0     9.847791e-01
...                    ...
537.0 538.0   7.086156e-03
      539.0   7.180250e-06
538.0 538.0   6.204500e-07
      539.0   2.573577e-07
539.0 539.0   3.349996e-10

[146070 rows x 1 columns]

In [101]:
f.index.names

FrozenList(['first', 'second'])

In [100]:
f.columns

Index(['predSuccess', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '531', '532', '533', '534', '535', '536', '537', '538', '539',
       'predResult'],
      dtype='object', length=542)

# PredTable

In [107]:
g = pd.read_csv('empowermentexploration\\data\\gametree\\20240925-1728\\alchemy2ElemPredTable-data-crawl300.csv', 'r')

C:\Users\chand\AppData\Local\Temp\ipykernel_4512\282004870.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  g = pd.read_csv('empowermentexploration\\data\\gametree\\20240925-1728\\alchemy2ElemPredTable-data-crawl300.csv', 'r')


In [112]:
g.index

RangeIndex(start=0, stop=519414, step=1)

In [119]:
g.shape

(519414, 4)

In [120]:
g

,fi,"st,second,t","ueSuccess,t","ueResult,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719"
0,"129.0,406.0,0.0,-1.0,0.00024453352100120725,0....",NaN,NaN,NaN
1,"52.0,590.0,0.0,-1.0,0.0013718720821464213,0.00...",NaN,NaN,NaN
2,"316.0,682.0,0.0,-1.0,0.0008520417521946168,0.0...",NaN,NaN,NaN
3,"406.0,627.0,0.0,-1.0,0.001569529184514724,0.00...",NaN,NaN,NaN
4,"468.0,483.0,0.0,-1.0,0.0028307569375697335,0.0...",NaN,NaN,NaN
...,...,...,...,...
519409,"319.0,374.0,0.0,-1.0,0.00040914607376941445,0....",NaN,NaN,NaN
519410,"588.0,631.0,0.0,-1.0,0.003932116110225437,0.00...",NaN,NaN,NaN
519411,"189.0,496.0,0.0,-1.0,0.007366603425577261,0.00...",NaN,NaN,NaN
519412,"127.0,163.0,0.0,-1.0,0.015295916296813745,0.00...",NaN,NaN,NaN


# Elements JSON file

In [121]:
import json
import pandas as pd

def read_json_file(file_path):
    """Read a JSON file and return its contents."""
    with open(file_path, 'r') as file:
        return json.load(file)

def display_json_structure(data, indent=0):
    """Recursively display the structure of a JSON object."""
    if isinstance(data, dict):
        for key, value in data.items():
            print('  ' * indent + str(key))
            display_json_structure(value, indent + 1)
    elif isinstance(data, list):
        print('  ' * indent + f'List with {len(data)} items')
        if data:
            display_json_structure(data[0], indent + 1)
    else:
        print('  ' * indent + f'Value: {type(data).__name__}')

def json_to_dataframe(data):
    """Convert JSON data to a pandas DataFrame if possible."""
    return pd.json_normalize(data)

def analyze_json(file_path):
    """Read and analyze a JSON file."""
    # Read the JSON file
    data = read_json_file(file_path)
    
    print("1. JSON Structure:")
    display_json_structure(data)
    
    print("\n2. JSON Content (first 5 items if list):")
    if isinstance(data, list):
        print(json.dumps(data[:5], indent=2))
    else:
        print(json.dumps(data, indent=2))
    
    print("\n3. Data Types:")
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{key}: {type(value)}")
    elif isinstance(data, list) and data:
        for key, value in data[0].items():
            print(f"{key}: {type(value)}")
    
    print("\n4. Converting to DataFrame:")
    try:
        df = json_to_dataframe(data)
        print(df.head())
        print("\nDataFrame Info:")
        print(df.info())
    except Exception as e:
        print(f"Could not convert to DataFrame: {e}")

# Usage
file_path = 'empowermentexploration\\resources\\littlealchemy\\data\\raw\\elements.JSON'
analyze_json(file_path)

1. JSON Structure:
rain,smoke
  List with 1 items
    Value: str
rain,smog
  List with 1 items
    Value: str
bird,metal
  List with 1 items
    Value: str
bird,steel
  List with 1 items
    Value: str
clock,sound
  List with 1 items
    Value: str
fruit,time
  List with 1 items
    Value: str
juice,time
  List with 1 items
    Value: str
plant,water
  List with 1 items
    Value: str
ocean,plant
  List with 2 items
    Value: str
plant,sea
  List with 2 items
    Value: str
life,space
  List with 1 items
    Value: str
dust,human
  List with 1 items
    Value: str
lizard,swamp
  List with 1 items
    Value: str
lizard,river
  List with 1 items
    Value: str
mountain,sheep
  List with 1 items
    Value: str
car,hospital
  List with 1 items
    Value: str
car,doctor
  List with 1 items
    Value: str
bird,human
  List with 1 items
    Value: str
fishing rod,human
  List with 1 items
    Value: str
grass,wild animal
  List with 1 items
    Value: str
sugar,wild animal
  List with 1 item

In [123]:
with open(file_path, 'r') as file:
    elements = json.load(file)

In [132]:
elements

{'rain,smoke': ['acid rain'],
 'rain,smog': ['acid rain'],
 'bird,metal': ['airplane'],
 'bird,steel': ['airplane'],
 'clock,sound': ['alarm clock'],
 'fruit,time': ['alcohol'],
 'juice,time': ['alcohol'],
 'plant,water': ['algae'],
 'ocean,plant': ['algae', 'seaweed'],
 'plant,sea': ['algae', 'seaweed'],
 'life,space': ['alien'],
 'dust,human': ['allergy'],
 'lizard,swamp': ['alligator'],
 'lizard,river': ['alligator'],
 'mountain,sheep': ['alpaca'],
 'car,hospital': ['ambulance'],
 'car,doctor': ['ambulance'],
 'bird,human': ['angel'],
 'fishing rod,human': ['angler'],
 'grass,wild animal': ['ant'],
 'sugar,wild animal': ['ant'],
 'desert,snow': ['antarctica'],
 'desert,ice': ['antarctica'],
 'glass,water': ['aquarium'],
 'fish,glass': ['aquarium'],
 'isle,isle': ['archipelago'],
 'armor,wild animal': ['armadillo'],
 'metal,tool': ['armor'],
 'steel,tool': ['armor'],
 'energy,volcano': ['ash', 'eruption'],
 'human,moon': ['astronaut'],
 'human,rocket': ['astronaut'],
 'human,space st